In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
x_train = pd.read_hdf('/content/drive/MyDrive/aes/t100/x_train.h5').values
x_train

array([[ 0.159101  ,  0.172359  ,  0.16352   , ...,  0.        ,
         0.00883892,  0.        ],
       [ 0.0331459 ,  0.0331459 ,  0.0331459 , ...,  0.0132584 ,
         0.0198876 ,  0.0110486 ],
       [ 0.159101  ,  0.159101  ,  0.185617  , ..., -0.00883892,
        -0.00883892,  0.0132584 ],
       ...,
       [ 0.159101  ,  0.150262  ,  0.16352   , ...,  0.00441946,
         0.        ,  0.        ],
       [ 0.024307  ,  0.0220973 ,  0.0265168 , ...,  0.0198876 ,
         0.0265168 ,  0.0220973 ],
       [ 0.0397751 ,  0.0441946 ,  0.0441946 , ...,  0.00441946,
         0.00883892,  0.        ]], dtype=float32)

In [5]:
x_test = pd.read_hdf('/content/drive/MyDrive/aes/t100/x_test.h5').values
x_test

array([[ 0.024307  ,  0.0265168 ,  0.0309362 , ...,  0.00220973,
         0.00441946,  0.        ],
       [ 0.00883892,  0.00662919,  0.00441946, ...,  0.00883892,
         0.00662919,  0.0132584 ],
       [ 0.0198876 ,  0.0309362 ,  0.0265168 , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.0309362 ,  0.0309362 ,  0.024307  , ..., -0.00220973,
        -0.00220973, -0.00441946],
       [ 0.0309362 ,  0.0287265 ,  0.0397751 , ...,  0.0220973 ,
         0.0198876 ,  0.0220973 ],
       [ 0.0220973 ,  0.0220973 ,  0.024307  , ...,  0.024307  ,
         0.0154681 ,  0.0110486 ]], dtype=float32)

In [6]:
y_train = pd.read_hdf('/content/drive/MyDrive/aes/t100/y_train.h5').values
y_train = to_categorical(y_train)
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [7]:
y_test = pd.read_hdf('/content/drive/MyDrive/aes/t100/y_test.h5').values
y_test = to_categorical(y_test)
y_test

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [8]:
from keras.backend import set_session
from keras.backend import clear_session
from keras.backend import get_session
import gc
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it does something you should see a number as output

    # use the same config as you used to create the session
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tf.compat.v1.Session(config=config))

In [9]:
# units : 라벨의 갯수, input_dim : 특성의 갯수
from tensorflow import keras
model = Sequential([
    # Dropout(rate = 0.5),
    Dense(units = 77, input_dim = 70000 , activation = 'softmax')
])

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])

callback = keras.callbacks.EarlyStopping(monitor='loss',min_delta=0.0001,patience=10,verbose=1,restore_best_weights=True)

In [10]:
reset_keras()
history = model.fit(x_train, y_train, batch_size = 512, epochs = 300, callbacks = [callback] ,validation_data = (x_test, y_test))

166
Epoch 1/300
110/110 [==============================] - 28s 209ms/step - loss: 2.0487 - acc: 0.5761 - val_loss: 1.2680 - val_acc: 0.7877
Epoch 2/300
110/110 [==============================] - 6s 51ms/step - loss: 1.0056 - acc: 0.8591 - val_loss: 0.8632 - val_acc: 0.8380
Epoch 3/300
110/110 [==============================] - 6s 51ms/step - loss: 0.7092 - acc: 0.9272 - val_loss: 0.6646 - val_acc: 0.9373
Epoch 4/300
110/110 [==============================] - 6s 52ms/step - loss: 0.5472 - acc: 0.9638 - val_loss: 0.5514 - val_acc: 0.9314
Epoch 5/300
110/110 [==============================] - 6s 52ms/step - loss: 0.4447 - acc: 0.9777 - val_loss: 0.4746 - val_acc: 0.9476
Epoch 6/300
110/110 [==============================] - 6s 52ms/step - loss: 0.3712 - acc: 0.9904 - val_loss: 0.4143 - val_acc: 0.9701
Epoch 7/300
110/110 [==============================] - 6s 51ms/step - loss: 0.3163 - acc: 0.9940 - val_loss: 0.3728 - val_acc: 0.9678
Epoch 8/300
110/110 [==============================] - 6

In [11]:
from sklearn.metrics import classification_report
y_pred = model.predict(x_test)
print(classification_report(np.argmax(y_test,axis=1),np.argmax(y_pred,axis=1)))

749/749 [==============================] - 3s 3ms/step
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       315
           2       0.98      0.98      0.98       315
           3       1.00      1.00      1.00       315
           4       1.00      1.00      1.00       315
           5       1.00      1.00      1.00       315
           6       1.00      1.00      1.00       316
           7       1.00      1.00      1.00       315
           8       1.00      1.00      1.00       315
           9       0.98      0.98      0.98       315
          10       1.00      1.00      1.00       315
          11       1.00      1.00      1.00       315
          12       1.00      1.00      1.00       315
          13       1.00      1.00      1.00       315
          14       1.00      1.00      1.00       315
          15       1.00      1.00      1.00       315
          16       1.00      1.00      1.00       315
          17       1.00   